In [3]:
%pip install gurobipy
import gurobipy as gp
modeloptions = {
"WLSACCESSID": '6bfc9135-9954-4699-9f70-8c9a8ac55999',
"WLSSECRET": 'daa4d80f-8378-4c86-a6fc-0eb78c022dd7',
"LICENSEID": 2564926,
}
env = gp.Env(params=modeloptions)

Set parameter WLSAccessID

Set parameter WLSSecret
Set parameter LicenseID to value 2564926
Academic license 2564926 - for non-commercial use only - registered to at___@vt.edu


In [1]:
import pandas as pd
import numpy as np
from gurobipy import GRB, quicksum
suppliers_data = pd.read_csv('TX_suppliers.csv').dropna()
hubs_data = pd.read_csv('TX_hubs_adjusted_with_tiers_and_counties.csv').dropna()
plants_data = pd.read_csv('TX_plants_with_fixed_cost_revenue.csv').dropna()
roads_data = pd.read_csv('TX_roads_adjusted.csv').dropna()
roads2_data = pd.read_csv('TX_roads2_adjusted.csv').dropna()
railroads_data = pd.read_csv('TX_railroads.csv').dropna()
network_data = pd.read_csv('TX_network.csv')
demand_data = pd.read_csv('TX_demand.csv')

In [2]:
suppliers_index = suppliers_data['county'].unique()
hubs_index = hubs_data['hub'].unique()
plants_index = plants_data['plant'].unique()
demand_index = demand_data['county'].unique()
model_outputs = {}

In [4]:
import matplotlib.pyplot as plt
import geopandas as gpd

def visualize_hubs_plants_with_save(
    hubs, plants, hubs_index, plants_index, v, w, shapefile_path, output_path
):
    """
    Visualizes the hubs and plants selected to open on a map and saves the visualization as an image.
    
    Args:
        hubs (pd.DataFrame): Hubs data.
        plants (pd.DataFrame): Plants data.
        hubs_index (list): List of hub indices.
        plants_index (list): List of plant indices.
        v (dict): Gurobi decision variable for hub openings.
        w (dict): Gurobi decision variable for plant openings.
        shapefile_path (str): Path to the shapefile for Texas.
        output_path (str): Path to save the output image.
    """
    # Load shapefile
    texas_map = gpd.read_file(shapefile_path)

    # Mark open hubs and plants
    hubs['open'] = [v[j].X for j in hubs_index]
    plants['open'] = [w[k].X for k in plants_index]

    # Filter open hubs and plants
    open_hubs = hubs[hubs['open'] > 0.5]
    open_plants = plants[plants['open'] > 0.5]

    # Create the map
    fig, ax = plt.subplots(figsize=(12, 8))
    texas_map.plot(ax=ax, color="lightgrey", edgecolor="black")

    # Plot hubs and plants
    if not open_hubs.empty:
        ax.scatter(
            open_hubs['longitude'],
            open_hubs['latitude'],
            color="blue",
            label="Open Hubs",
            zorder=5,
        )
    if not open_plants.empty:
        ax.scatter(
            open_plants['longitude'],
            open_plants['latitude'],
            color="green",
            label="Open Plants",
            zorder=5,
        )

    # Add legend and title
    ax.legend()
    ax.set_title("Visualization of Open Hubs and Plants", fontsize=16)
    ax.axis("off")

    # Save the visualization as an image
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    print(f"Visualization saved to {output_path}")
    plt.close(fig)

In [5]:
def analyze_objective_terms(model, suppliers_index, hubs_index, plants_index, demand_index,
                            cost_road, cost_railroads, cost_road2,
                            invest_cost_hub, invest_cost_bio,
                            cost_loading_truck_dot407, cost_loading_railcar,
                            penalty_cost, selling_price_bioethanol, selling_price_electricity):
        # Extract values of decision variables
        x_values = {(i, j): x[i, j].X for i in suppliers_index for j in hubs_index if x[i, j].X > 0}
        y_values = {(j, k): y[j, k].X for j in hubs_index for k in plants_index if y[j, k].X > 0}
        z_values = {(k, c): z[k, c].X for k in plants_index for c in demand_index if z[k, c].X > 0}
        u_values = {c: u[c].X for c in demand_index if u[c].X > 0}
        m_values = {(i, j): m[i, j].X for i in suppliers_index for j in hubs_index if m[i, j].X > 0}
        n_values = {(j, k): n[j, k].X for j in hubs_index for k in plants_index if n[j, k].X > 0}
        p_values = {(k, c): p[k, c].X for k in plants_index for c in demand_index if p[k, c].X > 0}
        e_values = {k: e[k].X for k in plants_index if e[k].X > 0}

        # Compute individual terms
        investment_cost_hubs = sum(invest_cost_hub[j] * v[j].X for j in hubs_index)
        investment_cost_plants = sum(invest_cost_bio[k] * w[k].X for k in plants_index)
        transport_cost_road = sum(cost_road[i, j] * x[i, j].X for i in suppliers_index for j in hubs_index)
        transport_cost_railroad = sum(cost_railroads[j, k] * y[j, k].X for j in hubs_index for k in plants_index)
        transport_cost_road2 = sum(cost_road2[k, c] * z[k, c].X for k in plants_index for c in demand_index) * bioethanol_density
        loading_cost_truck_dot407 = sum(cost_loading_truck_dot407 * m[i, j].X for i in suppliers_index for j in hubs_index)
        loading_cost_truck_dot406 = sum(cost_loading_truck_dot406 * p[k, c].X for k in plants_index for c in demand_index)
        loading_cost_rail = sum(cost_loading_railcar * number_of_railcars * n[j, k].X for j in hubs_index for k in plants_index)
        unmet_demand_penalty = penalty_cost * sum(u[c].X for c in demand_index)
        revenue_bioethanol = selling_price_bioethanol * sum(z[k, c].X for k in plants_index for c in demand_index)
        revenue_electricity = selling_price_electricity * sum(e[k].X for k in plants_index)

        # Print individual terms
        print("### Objective Function Terms ###")
        print(f"Investment Cost - Hubs: ${investment_cost_hubs:.2f}")
        print(f"Investment Cost - Plants: ${investment_cost_plants:.2f}")
        print(f"Transportation Cost - Road: ${transport_cost_road:.2f}")
        print(f"Transportation Cost - Railroad: ${transport_cost_railroad:.2f}")
        print(f"Transportation Cost - Road2 (Plants to Counties): ${transport_cost_road2:.2f}")
        print(f"Loading Cost - Truck (DOT 407): ${loading_cost_truck_dot407:.2f}")
        print(f"Loading Cost - Truck (DOT 406): ${loading_cost_truck_dot406:.2f}")
        print(f"Loading Cost - Rail: ${loading_cost_rail:.2f}")
        print(f"Unmet Demand Penalty: ${unmet_demand_penalty:.2f}")
        print(f"Revenue - Bioethanol: ${revenue_bioethanol:.2f}")
        print(f"Revenue - Electricity: ${revenue_electricity:.2f}")

        # Compute total objective value manually
        total_cost = (
            (revenue_bioethanol + revenue_electricity) - (investment_cost_hubs +
            investment_cost_plants +
            transport_cost_road +
            transport_cost_railroad +
            transport_cost_road2 +
            loading_cost_truck_dot407 +
            loading_cost_truck_dot406 +
            loading_cost_rail +
            unmet_demand_penalty)
        )

        # Print total and compare with model objective value
        print(f"\n### Total Objective Value ###")
        print(f"Total Objective Value (Computed): ${total_cost:.2f}")
        print(f"Total Objective Value (Model): ${model.ObjVal:.2f}")

In [6]:
with gp.Env(params=modeloptions) as env, gp.Model(env=env) as model:
    #Parameters
    supplier_biomass = suppliers_data.set_index('county')['supply']
    cost_road = roads_data.set_index(['county', 'hub'])['adjusted_cost']
    cost_road2 = roads2_data.set_index(['plant', 'county'])['adjusted_cost']
    cost_railroads = railroads_data.set_index(['hub', 'plant'])['cost']
    invest_cost_hub = hubs_data.set_index('hub')['adjusted_investment_cost']
    invest_cost_bio = plants_data.set_index('plant')['adjusted_investment_cost']
    cost_loading_railcar = 854.60 #$
    capacity_hub = hubs_data.set_index('hub')['capacity']
    capacity_plant = plants_data.set_index('plant')['capacity']
    number_of_railcars = 100 
    capacity_railcar = 85.5 #Mg
    conversion_yield = plants_data.set_index('plant')['yield']
    cost_loading_truck_dot407 = 143.22 #$
    cost_loading_truck_dot406 = 161.16 * 0.789 #$
    capacity_truck_dot407 = 23.8 #Mg
    capacity_truck_dot406 = 26.8 #Mg
    penalty_cost = 0.5 #Penalty Per Liter
    selling_price_bioethanol = 0.48 #dollar per liter
    selling_price_electricity = 45.1 #dollar per MWh
    bioethanol_density = 0.000789 #Mg/L
    county_demand = demand_data.set_index('county')['demand']
    electricity_bio = plants_data.set_index('plant')['electricity_generated_mwh']
    electricity_yield_per_mg =  0.4725 # MWh/Mg
    total_electricity_potential = 60000  # MWh
    
    # Decision variables
    x = model.addVars(suppliers_index, hubs_index, vtype=GRB.CONTINUOUS, lb = 0, name="flow_county_hub")  # Flow from counties to hubs
    y = model.addVars(hubs_index, plants_index, vtype=GRB.CONTINUOUS, lb =0, name="flow_hub_plant")  # Flow from hubs to plants
    v = model.addVars(hubs_index, vtype=GRB.BINARY, name="open_hub")  # Whether to open a hub
    w = model.addVars(plants_index, vtype=GRB.BINARY, name="open_plant")  # Whether to open a biorefinery
    n = model.addVars(hubs_index, plants_index, vtype=GRB.INTEGER, lb =0, name="number of trains")  # Number of trains from hubs to plants
    z = model.addVars(plants_index, demand_index, vtype= GRB.CONTINUOUS, lb = 0, name = "flow_bio_county") #Flow from bioref to county
    u = model.addVars(demand_index, vtype = GRB.CONTINUOUS, name = "unmet_demand") #Unmet demand
    m = model.addVars(suppliers_index, hubs_index, vtype = GRB.INTEGER, lb = 0, name = "number_trucks_1") #Number of trucks to move from supplier to hub
    p = model.addVars(plants_index, demand_index, vtype = GRB.INTEGER, lb = 0, name = "number_trucks_2") #Number of trucks to move from plants to counties
    e = model.addVars(plants_index, vtype=GRB.CONTINUOUS, lb=0, name="electricity_generated")

    #Constraints
    supply_constraints = model.addConstrs(
        (quicksum(x[i, j] for j in hubs_index) <= supplier_biomass[i] for i in suppliers_index),
        name="supply_constraints") #Limit of possible supply

    demand_satisfaction_county = model.addConstrs(
        (quicksum(z[k, c] for k in plants_index) + u[c] >= county_demand[c] for c in demand_index),
        name="demand_satisfaction_county") #Required demand of county

    train_capacity_constraint = model.addConstrs(
        (n[j, k] * capacity_railcar * number_of_railcars >= y[j, k] for j in hubs_index for k in plants_index),
        name="train_capacity_constraint") #Limit of train capacity

    truck_dot407_capacity = model.addConstrs(
        (m[i, j] * capacity_truck_dot407 >= x[i, j] for j in hubs_index for i in suppliers_index),
        name="truck1_capacity_constraint") #Limit of truck capacity

    truck_dot406_capacity = model.addConstrs(
        (p[k, c] * capacity_truck_dot406 >= z[k, c] * bioethanol_density for k in plants_index for c in demand_index),
        name="truck2_capacity_constraint") #Limit of truck capacity

    flow_conservation_constraint = model.addConstrs(
        (quicksum(x[i, j] for i in suppliers_index) == quicksum(y[j, k] for k in plants_index) for j in hubs_index),
        name="flow_conservation_constraint") #Conversion of inputs to outputs from hubs

    hub_capacity_constraint = model.addConstrs(
        (quicksum(x[i, j] for i in suppliers_index) <= capacity_hub[j] * v[j] for j in hubs_index),
        name="hub_capacity_constraint") #Limit fo hub capacity

    plant_capacity_constraint = model.addConstrs(
        (quicksum(y[j, k] for j in hubs_index) <= capacity_plant[k] * w[k] / conversion_yield[k] for k in plants_index),
        name="plant_capacity_constraint") #Limit of biorefinery capacity

    flow_conservation_constraint = model.addConstrs(
        (quicksum(z[k, c] for c in demand_index) == 232 * quicksum(y[j, k] for j in hubs_index) for k in plants_index),
        name="flow_conservation_constraint") #Conversion of inputs to outputs from hubs
    
    electricity_generated_constraint = model.addConstrs(
        (e[k] == quicksum(y[j, k] for j in hubs_index) * electricity_yield_per_mg for k in plants_index), 
        name="electricity_generated_constraint")
    
    electricity_potential_constraint = model.addConstrs(
        (e[k] <= total_electricity_potential for k in plants_index),
        name="electricity_potential_constraint")

    #Objective Function
    model.setObjective(selling_price_bioethanol * quicksum(z[k,c] for k in plants_index for c in demand_index) + selling_price_electricity * quicksum(e[k] for k in plants_index) - (quicksum(invest_cost_hub[j] * v[j] for j in hubs_index) + quicksum(invest_cost_bio[k] * w[k] for k in plants_index) + quicksum(cost_road[i, j] * x[i, j] for i in suppliers_index for j in hubs_index) + quicksum(cost_railroads[j,k] * y[j, k] for j in hubs_index for k in plants_index) + quicksum(cost_road2[k, c] * z[k, c] for k in plants_index for c in demand_index) * bioethanol_density  + quicksum(cost_loading_railcar * number_of_railcars * n[j, k] for j in hubs_index for k in plants_index) + quicksum(cost_loading_truck_dot407 * m[i,j] for i in suppliers_index for j in hubs_index) + quicksum(cost_loading_truck_dot406 * p[k,c] for k in plants_index for c in demand_index) + penalty_cost * quicksum(u[c] for c in demand_index)), GRB.MAXIMIZE)
    model.setParam('MIPGap', 0.006)   # Stop at 1% optimality gap    model.write("model.lp")  # Save model in LP format
    model.write("model.mps")  # Save model in MPS format
    model.optimize()
    # Run optimization with callback
    # Analyze objective terms after optimization
    if model.Status in [GRB.OPTIMAL, GRB.SUBOPTIMAL, GRB.INTERRUPTED]:
        analyze_objective_terms(
            model=model,
            suppliers_index=suppliers_index,
            hubs_index=hubs_index,
            plants_index=plants_index,
            demand_index=demand_index,
            cost_road=cost_road,
            cost_railroads=cost_railroads,
            cost_road2=cost_road2,
            invest_cost_hub=invest_cost_hub,
            invest_cost_bio=invest_cost_bio,
            cost_loading_truck_dot407=cost_loading_truck_dot407,
            cost_loading_railcar=cost_loading_railcar,
            penalty_cost=penalty_cost,
            selling_price_bioethanol=selling_price_bioethanol,
            selling_price_electricity=selling_price_electricity
        )
        print(f"Optimal Objective Value: {model.ObjVal}\n")

        # Printing variable values
        print("\n### Summary of Results ###")
    
        # Total number of open hubs
        total_hubs = sum(1 for j in hubs_index if v[j].X > 0)
        print(f"Total Open Hubs: {total_hubs}")
        
        # Total number of open plants
        total_plants = sum(1 for k in plants_index if w[k].X > 0)
        print(f"Total Open Plants: {total_plants}")
        
        # Total trucks from suppliers to hubs (m)
        total_trucks_m = sum(m[i, j].X for i in suppliers_index for j in hubs_index if m[i, j].X > 0)
        print(f"Total Trucks (Supplier to Hub): {total_trucks_m}")
        
        # Total trucks from plants to counties (p)
        total_trucks_p = sum(p[k, c].X for k in plants_index for c in demand_index if p[k, c].X > 0)
        print(f"Total Trucks (Plant to County): {total_trucks_p}")
        
        # Total trains from hubs to plants
        total_trains = sum(n[j, k].X for j in hubs_index for k in plants_index if n[j, k].X > 0)
        print(f"Total Trains (Hub to Plant): {total_trains}")
        
        # Total unmet demand
        total_unmet_demand = sum(u[c].X for c in demand_index if u[c].X > 0)
        print(f"Total Unmet Demand (Liters): {total_unmet_demand}")
        
        # Total electricity generated
        total_electricity_generated = sum(e[k].X for k in plants_index if e[k].X > 0)
        print(f"Total Electricity Generated (MWh): {total_electricity_generated}")
        # Call function
        visualize_hubs_plants_with_save(
            hubs=hubs_data,  # Your hubs DataFrame
            plants=plants_data,  # Your plants DataFrame
            hubs_index=hubs_index,  # List of hub indices
            plants_index=plants_index,  # List of plant indices
            v=v,  # Gurobi decision variable for hub openings
            w=w,  # Gurobi decision variable for plant openings
            shapefile_path="Tx_CntyBndry_Detail_TIGER500k.shp",  # Path to Texas shapefile
            output_path="hubs_plants_visualization.png"  # Path to save the output image
        )
        # Populate model_outputs dictionary
        model_outputs['objective_value'] = model.ObjVal
        model_outputs['decision_variables'] = {
            var.VarName: var.X for var in model.getVars() if var.X > 0
        }
        model_outputs['hubs_to_open'] = [j for j in hubs_index if v[j].X > 0.5]
        model_outputs['plants_to_open'] = [k for k in plants_index if w[k].X > 0.5]
        model_outputs['total_trucks'] = sum(m[i, j].X for i in suppliers_index for j in hubs_index if m[i, j].X > 0)
        model_outputs['total_trains'] = sum(n[j, k].X for j in hubs_index for k in plants_index if n[j, k].X > 0)
    else:
        print("Optimization did not produce a feasible solution.")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2564926
Academic license 2564926 - for non-commercial use only - registered to at___@vt.edu
Set parameter MIPGap to value 0.006
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 8840HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2564926 - for non-commercial use only - registered to at___@vt.edu
Optimize a model with 57553 rows, 113243 columns and 245436 nonzeros
Model fingerprint: 0xd8048632
Variable types: 56732 continuous, 56511 integer (200 binary)
Coefficient statistics:
  Matrix range     [8e-04, 1e+06]
  Objective range  [3e-01, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+08]
Found heuristic solution: objective -3.64192e+08
Found heuristic solution: objective -3.64192e+08
Presolve removed 371 rows and 3

In [7]:
# Example of accessing the model_outputs
print("Objective Value:", model_outputs['objective_value'])
print("Hubs to Open:", model_outputs['hubs_to_open'])
print("Plants to Open:", model_outputs['plants_to_open'])
print("Total Trucks:", model_outputs['total_trucks'])
print("Total Trains:", model_outputs['total_trains'])

Objective Value: 119674626.71532619
Hubs to Open: [17952, 17945, 17466, 18042, 17934, 17359, 17201, 17592, 18127, 18303, 17620]
Plants to Open: [10062, 10059, 10060, 9133, 9142, 9132, 10056, 10058, 10066, 9131, 9085, 9107, 9140, 9174, 9057, 9054, 9056, 9053, 9060, 9184, 9204, 9040, 9105, 9088]
Total Trucks: 128136.0
Total Trains: 360.0


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

def filter_and_visualize(
    map_path, hubs_file, plants_file, demand_file, hubs_to_open, plants_to_open, save_path=None
):
    """
    Filter open hubs and plants, add county names, and visualize on the Texas map.
    Also, print details of open hubs and plants.

    Args:
        map_path (str): Path to the Texas shapefile for the map.
        hubs_file (str): Path to the hubs data file.
        plants_file (str): Path to the plants data file.
        demand_file (str): Path to the demand data file (for county names).
        hubs_to_open (list): List of hub IDs to open.
        plants_to_open (list): List of plant IDs to open.
        save_path (str, optional): Path to save the plot as an image. If None, the plot is shown but not saved.
    """
    # Load the data
    hubs_data = pd.read_csv(hubs_file)
    plants_data = pd.read_csv(plants_file)
    demand_data = pd.read_csv(demand_file)

    # Map FIPS to county names
    county_map = demand_data.set_index("county")[["CNTY_NM"]]

    # Filter open hubs and plants
    hubs_data["open"] = hubs_data["hub"].isin(hubs_to_open).astype(int)
    plants_data["open"] = plants_data["plant"].isin(plants_to_open).astype(int)

    hubs_open_details = hubs_data[hubs_data["open"] == 1]
    plants_open_details = plants_data[plants_data["open"] == 1]

    # Add county names
    hubs_open_details = hubs_open_details.merge(county_map, left_on="county", right_index=True, how="left")
    plants_open_details = plants_open_details.merge(county_map, left_on="county", right_index=True, how="left")

    # Print details
    print("### Open Hubs Details ###")
    print(hubs_open_details[["hub", "CNTY_NM", "latitude", "longitude", "tier", "capacity"]])

    print("\n### Open Plants Details ###")
    print(plants_open_details[["plant", "CNTY_NM", "latitude", "longitude", "tier", "capacity"]])

    # Load the Texas map
    texas_map = gpd.read_file(map_path)

    # Create GeoDataFrames for hubs and plants
    hubs_gdf = gpd.GeoDataFrame(
        hubs_open_details,
        geometry=gpd.points_from_xy(hubs_open_details["longitude"], hubs_open_details["latitude"]),
        crs="EPSG:4326"
    )

    plants_gdf = gpd.GeoDataFrame(
        plants_open_details,
        geometry=gpd.points_from_xy(plants_open_details["longitude"], plants_open_details["latitude"]),
        crs="EPSG:4326"
    )

    # Define a common scaling factor to match medium hub and medium plant sizes
    hub_scale_factor = 300000
    plant_scale_factor = 152063705
    common_scale = 200  # Adjust as necessary to ensure appropriate marker sizes

    hubs_gdf["scaled_capacity"] = hubs_gdf["capacity"] / hub_scale_factor * common_scale
    plants_gdf["scaled_capacity"] = plants_gdf["capacity"] / plant_scale_factor * common_scale

    # Plot the map
    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    texas_map.plot(ax=ax, color="lightgrey", edgecolor="black", alpha=0.5)

    # Plot hubs
    hubs_gdf.plot(
        ax=ax,
        color="blue",
        markersize=hubs_gdf["scaled_capacity"],
        label="Hubs (size by tier)",
        alpha=0.6
    )

    # Plot plants
    plants_gdf.plot(
        ax=ax,
        color="green",
        markersize=plants_gdf["scaled_capacity"],
        label="Plants (size by tier)",
        alpha=0.6
    )

    # Add legend and title
    plt.legend()
    plt.title("Open Hubs and Plants in Texas (Size by Tier)", fontsize=14)

    # Save or show the plot
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    else:
        plt.show()
    plt.close()

    print("Filtered and Visualized Open Hubs and Plants.")

# Example Usage
filter_and_visualize(
    map_path="Tx_CntyBndry_Detail_TIGER500k.shp",  # Path to the Texas map shapefile
    hubs_file="TX_hubs_adjusted_with_tiers_and_counties.csv",  # Path to hubs file
    plants_file="TX_plants_with_fixed_cost_revenue.csv",  # Path to plants file
    demand_file="TX_demand.csv",  # Path to demand data file
    hubs_to_open=model_outputs['hubs_to_open'],  # Replace with your data
    plants_to_open=model_outputs['plants_to_open'],  # Replace with your data
    save_path="open_hubs_plants_map.png"  # Set to None if you don't want to save
)


### Open Hubs Details ###
      hub       CNTY_NM  latitude  longitude    tier  capacity
1   17952       Coleman  31.83528  -99.42767   Small     75000
3   17945     Wilbarger  34.16258  -99.28511   Small     75000
8   17466         Jones  32.61514  -99.81358   Small     75000
11  18042       Fayette  29.67922  -96.90505  Medium    300000
13  17934    Palo Pinto  32.66522  -98.11879  Medium    300000
17  17359       Wharton  29.31990  -96.10283  Medium    300000
19  17201         Lamar  33.64844  -95.56841  Medium    300000
20  17592       Navarro  32.09099  -96.46175  Medium    300000
21  18127  San Patricio  28.03355  -97.50889  Medium    300000
22  18303       Coryell  31.12247  -97.89691   Large    600000
24  17620       Wichita  33.85992  -98.59059   Large    600000

### Open Plants Details ###
    plant     CNTY_NM   latitude   longitude   tier  capacity
0   10062        Gray  35.483894 -101.051743  Small  38015926
1   10059  Hutchinson  35.699722 -101.360000  Small  38015926
2  